In [1]:
import os

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Função para carregar e processar os dados de cada modelo
def process_model_data(csv_path, model_name):
    data = pd.read_csv(csv_path)
    # Adiciona a identificação do modelo nos dados
    data["model"] = model_name
    return data

# Função para calcular métricas estatísticas
def calculate_metrics(data, metrics, model_name):
    stats = {}
    for metric in metrics:
        stats[f"{metric}_mean"] = data[metric].mean()
        stats[f"{metric}_std"] = data[metric].std()
        stats[f"{metric}_max"] = data[metric].max()
        stats[f"{metric}_min"] = data[metric].min()
    stats["model"] = model_name
    return stats

# Função para salvar métricas como CSV
def save_metrics_as_csv(metrics, filename):
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv(filename, index=False)
    print(f"Métricas salvas em: {filename}")

# Função para gerar e salvar gráficos
def save_metric_plot(data_general, data_subdivided, metric, ylabel, title, filename):
    plt.figure(figsize=(10, 6))
    plt.plot(data_general["epoch"], data_general[metric], label="Modelo Geral")
    plt.plot(data_subdivided["epoch"], data_subdivided[metric], label="Modelo Subdividido", linestyle='--')
    plt.xlabel("Época")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(filename)
    print(f"Gráfico salvo em: {filename}")
    plt.close()

In [3]:

from pathlib import Path


# Função principal
def main():
    def get_path_from_cwd(x): return Path(os.path.join(os.getcwd(), "..", x)).resolve()

    # Configurações de arquivos
    general_csv = get_path_from_cwd("runs/detect/train23/results.csv")
    subdivided_csv = get_path_from_cwd("runs/detect/train21/results.csv")
    output_folder = get_path_from_cwd("experiments/comparison")

    os.makedirs(output_folder, exist_ok=True)

    # Carregar os dados
    data_general = process_model_data(general_csv, "Geral")
    data_subdivided = process_model_data(subdivided_csv, "Subdividido")

    # Definir métricas de interesse
    metrics = ["metrics/mAP50(B)", "metrics/mAP50-95(B)", "metrics/precision(B)", "metrics/recall(B)",
               "train/box_loss", "train/cls_loss", "val/box_loss", "val/cls_loss"]

    # Calcular estatísticas gerais para cada modelo
    general_metrics = calculate_metrics(data_general, metrics, "Geral")
    subdivided_metrics = calculate_metrics(data_subdivided, metrics, "Subdividido")

    # Salvar estatísticas em CSV
    save_metrics_as_csv(general_metrics, os.path.join(output_folder, "general_metrics_summary.csv"))
    save_metrics_as_csv(subdivided_metrics, os.path.join(output_folder, "subdivided_metrics_summary.csv"))

    # Gerar gráficos e salvar como PNG
    for metric in metrics:
        ylabel = metric.split("/")[-1]  # Nome mais simples para o eixo Y
        plot_title = f"Comparação de {ylabel} entre os Modelos"
        plot_filename = os.path.join(output_folder, f"{metric.replace('/', '_')}_comparison.png")
        save_metric_plot(data_general, data_subdivided, metric, ylabel, plot_title, plot_filename)

    # Salvar métricas finais da última época
    general_final_metrics = data_general.iloc[-1][metrics].to_dict()
    subdivided_final_metrics = data_subdivided.iloc[-1][metrics].to_dict()
    final_metrics = pd.DataFrame([general_final_metrics, subdivided_final_metrics])
    final_metrics["model"] = ["Geral", "Subdividido"]
    final_metrics.to_csv(os.path.join(output_folder, "final_metrics_comparison.csv"), index=False)
    print(f"Métricas finais salvas em: {os.path.join(output_folder, 'final_metrics_comparison.csv')}")

In [4]:

if __name__ == "__main__":
    main()

Métricas salvas em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/general_metrics_summary.csv
Métricas salvas em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/subdivided_metrics_summary.csv
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/metrics_mAP50(B)_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/metrics_mAP50-95(B)_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/metrics_precision(B)_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/metrics_recall(B)_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/train_box_loss_comparison.png
Gráfico salvo em: /mnt/c/Users/Ricardo/Documents/Development/tcc-1/experiments/comparison/train_cls_loss_comparison.png
Gráfico salvo em: /mn